In [1]:
import glob
import os

import cv2
import numpy as np
import pydicom

In [2]:
def save_pic(addr, mr_slice, src, thresh):
    lower = np.array([thresh])  # -- Lower range --
    upper = np.array([255])  # -- Upper range --
    mask = cv2.inRange(src, lower, upper)
    res = cv2.bitwise_and(src, src, mask=mask)
    cv2.imwrite(addr + str(mr_slice) + '.png', res)



In [3]:
def save_pic2(addr, mr_slice, src, thresh):
    lower = np.array([0])  # -- Lower range --
    upper = np.array([thresh])  # -- Upper range --
    mask = cv2.inRange(src, lower, upper)
    res = cv2.bitwise_and(src, src, mask=mask)
    cv2.imwrite(addr + str(mr_slice) + '.png', res)


In [4]:
def save_pic3(addr, mr_slice, src, thresh):
    lower = np.array([0])  # -- Lower range --
    upper = np.array([thresh])  # -- Upper range --
    mask = cv2.inRange(src, lower, upper)
    res = cv2.bitwise_and(src, src, mask=mask)

    lower = np.array([thresh])  # -- Lower range --
    upper = np.array([255])  # -- Upper range --
    mask = cv2.inRange(src, lower, upper)
    new_src = cv2.morphologyEx(res, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11)))
    res_close = cv2.bitwise_and(new_src, new_src, mask=mask)

    cv2.imwrite(addr + str(mr_slice) + '.png', res + res_close)

In [5]:
def _manual_otsu(img, ret):
    new_img = []
    for pixel_list in img:
        for pixel in pixel_list:
            if pixel > ret:
                new_img.append(pixel)

    new_img = np.array(new_img)
    hist = cv2.calcHist([new_img], [0], None, [256], [0, 256])
    hist_norm = hist.ravel() / hist.max()
    q = hist_norm.cumsum()
    bins = np.arange(256)
    fn_min = np.inf
    thresh = -1

    for j in range(1, 256):
        p1, p2 = np.hsplit(hist_norm, [j])  # probabilities
        q1, q2 = q[j], q[255] - q[j]  # cum sum of classes
        b1, b2 = np.hsplit(bins, [j])  # weights
        # finding means and variances
        m1, m2 = np.sum(p1 * b1) / q1, np.sum(p2 * b2) / q2
        v1, v2 = np.sum(((b1 - m1) ** 2) * p1) / q1, np.sum(((b2 - m2) ** 2) * p2) / q2
        # calculates the minimization function
        fn = v1 * q1 + v2 * q2
        if fn < fn_min:
            fn_min = fn
            thresh = j
    return thresh, new_img.shape


def color_detection(img, addr, mr_slice):
    first_thresh, imgf = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    thresh = first_thresh
    i = 0

    while i < 3:
        sec_thresh, shape = _manual_otsu(img, thresh)
        if i == 2:
            save_pic3(addr, mr_slice, img, sec_thresh)
        thresh = sec_thresh
        i += 1

    return first_thresh



In [6]:
dataDir = "../data/Dataset/"
destDir = "../data/Preprocessed_Dataset/"
try:
    os.mkdir(destDir)
except FileExistsError:
    for f in glob.glob(destDir):
        os.remove(f)
patients = os.listdir(dataDir)
n_patients = os.listdir(destDir)
patients.sort()
new_patient = 1

for patient in patients:
    slices = []
    path = dataDir + str(patient)
    os.mkdir(destDir + str(patient))
    for file in os.listdir(path):
        slices.append((pydicom.dcmread(os.path.join(path, file)).pixel_array.astype(float), file))
    for s in range(len(slices)):
        color_detection(np.array(slices[s][0] * 255.0 / np.max(slices[s][0]), dtype=np.uint8),
                        destDir + str(patient) + '/', int(slices[s][1]))

/Users/reza/PycharmProjects/Medical-Imaging/venv/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/Users/reza/PycharmProjects/Medical-Imaging/venv/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in double_scalars
/Users/reza/PycharmProjects/Medical-Imaging/venv/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
